# Day 19: Medicine for Rudolph

https://adventofcode.com/2015/day/19

## Part 1

In [2]:
from functools import lru_cache

@lru_cache
def stringToElements(s):
    sv = []
    i = 0
    while i<len(s):
        if i<len(s)-1 and s[i+1].islower():
            sv.append(s[i:i+2])
            i+=2
        else:
            sv.append(s[i])
            i+=1
    return sv

def getMolAndRepl(filename):
    f = open(filename)
    r,s = f.read().split("\n\n")
    s = s.strip('\n')
    repl = {}
    for l in r.split('\n'):
        t = l.strip("\n").split(" => ")
        if t[0] in repl.keys():
            repl[t[0]].append( stringToElements(t[1]) )
        else:
            repl[t[0]] = [ stringToElements(t[1]) ]
    return s,repl

In [67]:
print(stringToElements("NRnFAr"))
print(stringToElements("NRnFAr"))

['N', 'Rn', 'F', 'Ar']
['N', 'Rn', 'F', 'Ar']


In [68]:
#s, repl = getMolAndRepl("./data/day19test0.txt")
s, repl = getMolAndRepl("./data/input19.txt")
s, len(s), repl

('CRnCaCaCaSiRnBPTiMgArSiRnSiRnMgArSiRnCaFArTiTiBSiThFYCaFArCaCaSiThCaPBSiThSiThCaCaPTiRnPBSiThRnFArArCaCaSiThCaSiThSiRnMgArCaPTiBPRnFArSiThCaSiRnFArBCaSiRnCaPRnFArPMgYCaFArCaPTiTiTiBPBSiThCaPTiBPBSiRnFArBPBSiRnCaFArBPRnSiRnFArRnSiRnBFArCaFArCaCaCaSiThSiThCaCaPBPTiTiRnFArCaPTiBSiAlArPBCaCaCaCaCaSiRnMgArCaSiThFArThCaSiThCaSiRnCaFYCaSiRnFYFArFArCaSiRnFYFArCaSiRnBPMgArSiThPRnFArCaSiRnFArTiRnSiRnFYFArCaSiRnBFArCaSiRnTiMgArSiThCaSiThCaFArPRnFArSiRnFArTiTiTiTiBCaCaSiRnCaCaFYFArSiThCaPTiBPTiBCaSiThSiRnMgArCaF',
 505,
 {'Al': [['Th', 'F'], ['Th', 'Rn', 'F', 'Ar']],
  'B': [['B', 'Ca'], ['Ti', 'B'], ['Ti', 'Rn', 'F', 'Ar']],
  'Ca': [['Ca', 'Ca'],
   ['P', 'B'],
   ['P', 'Rn', 'F', 'Ar'],
   ['Si', 'Rn', 'F', 'Y', 'F', 'Ar'],
   ['Si', 'Rn', 'Mg', 'Ar'],
   ['Si', 'Th']],
  'F': [['Ca', 'F'], ['P', 'Mg'], ['Si', 'Al']],
  'H': [['C', 'Rn', 'Al', 'Ar'],
   ['C', 'Rn', 'F', 'Y', 'F', 'Y', 'F', 'Ar'],
   ['C', 'Rn', 'F', 'Y', 'Mg', 'Ar'],
   ['C', 'Rn', 'Mg', 'Y', 'F', 'Ar'],
   ['H', 'Ca'],
   ['

In [69]:
from collections import defaultdict

def replaceElements(s,repl):
    sv = stringToElements(s)
    reps = defaultdict(lambda:0)
    for i in range(len(sv)):
        repi = []
        if sv[i] in repl.keys():
            for r in repl[sv[i]]:
                repi.append(sv[:i]+r+sv[i+1:])
        else:
            repi.append(sv)
        for r in repi:
             if r != sv: 
                reps["".join(r)] += 1
    return reps

s, repl = getMolAndRepl("./data/day19test0.txt")
reps = replaceElements(s,repl)
print("Part 1 Test    = ",len(reps))

s, repl = getMolAndRepl("./data/input19.txt")
reps = replaceElements(s,repl)
print("Part 1 Full    = ",len(reps))

Part 1 Test    =  4
Part 1 Full    =  535


## Part 2. First (failed) attempt: some kind of BFS algorithm

Breadth-First Search algorithm (I think), with some pruning of solutions that could not complete to final molecule added. Unfortunately, it does not converge in time (list of partial molecules to expand becomes too large).

In [70]:
from copy import deepcopy

#s, repl = getMolAndRepl("./data/input19.txt")

s, repl = getMolAndRepl("./data/day19test1.txt")
s = "HOHOHO"

print("Target molecule element lenght =",len(stringToElements(s)))

mol = {'e' : 1}

steps = 0
sv = stringToElements(s)

print("Evolving molecules...")
print("Step # Molecules")

while s not in mol.keys():
    
    newmol = defaultdict(lambda:0)
    
    for m in mol.keys():
        
        for r in replaceElements(m,repl):
            
            addToNext = True

            # remove if molecule component is different from target
            # component, and element cannot be replaced
            if addToNext:
                rv = stringToElements(r)
                for e1,e2 in zip(rv,sv[:len(rv)]):
                    if e1!=e2 and e1 not in repl.keys(): 
                        addToNext = False
                        break
            
            if addToNext:
                newmol[r] += 1
                
    mol = deepcopy(newmol)
    steps +=1 
    print(steps, len(mol))

print("Target Molecule found at step =",steps)

Target molecule element lenght = 6
Evolving molecules...
Step # Molecules
1 2
2 3
3 7
4 15
5 31
6 63
Target Molecule found at step = 6


## Part 2: Second (successfull!) attempt: going backward...

Idea: 
* start from target molecule
* count how many *replacement* molecules it contains, replace with origin element
* rinse and repeat

In [24]:
s, repl = getMolAndRepl("./data/input19.txt")

print("Target molecule lenght =",len(s))

count = 0
while len(s)>1:
    for k in repl.keys():
        for g in repl[k]:
            sg = "".join(g) # replacement molecule corresponsing to element k
            if sg in s:
                count += s.count(sg) # it can be containes more then once!
                s = s.replace(sg,k)
                print("Replacing",count,"molecules -> New molecule lenght =",len(s))
                
print("Final molecule =",s,", obtained with",count,"steps")

Target molecule lenght = 505
Replacing 2 molecules -> New molecule lenght = 503
Replacing 3 molecules -> New molecule lenght = 498
Replacing 7 molecules -> New molecule lenght = 490
Replacing 15 molecules -> New molecule lenght = 474
Replacing 16 molecules -> New molecule lenght = 468
Replacing 25 molecules -> New molecule lenght = 450
Replacing 37 molecules -> New molecule lenght = 450
Replacing 41 molecules -> New molecule lenght = 434
Replacing 44 molecules -> New molecule lenght = 413
Replacing 48 molecules -> New molecule lenght = 389
Replacing 63 molecules -> New molecule lenght = 359
Replacing 73 molecules -> New molecule lenght = 339
Replacing 75 molecules -> New molecule lenght = 335
Replacing 79 molecules -> New molecule lenght = 323
Replacing 83 molecules -> New molecule lenght = 323
Replacing 86 molecules -> New molecule lenght = 317
Replacing 89 molecules -> New molecule lenght = 311
Replacing 92 molecules -> New molecule lenght = 305
Replacing 99 molecules -> New molecule